In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorflow import keras
from keras.layers import Input, Dense, Dropout, LeakyReLU
from keras.models import Model, Sequential
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import initializers

In [3]:
np.random.seed(10)
random_dim=100

In [4]:
def load_mnist_data():
  (x_train, y_train), (x_test, y_test)=mnist.load_data()
  x_train=(x_train.astype(np.float32)-127.5)/127.5
  x_train=x_train.reshape(60000, 784)
  return (x_train, y_train, x_test, y_test)

In [5]:
def get_optimizer():
  return Adam(learning_rate=0.0002, beta_1=0.5)

In [6]:
def get_generator(optimizer):
  generator=Sequential()
  generator.add(Dense(256, input_dim=random_dim, \
                      kernel_initializer=initializers.RandomNormal(stddev=0.02)))
  generator.add(LeakyReLU(0.2))

  generator.add(Dense(512))
  generator.add(LeakyReLU(0.2))

  generator.add(Dense(1024))
  generator.add(LeakyReLU(0.2))

  generator.add(Dense(784, activation='tanh'))
  generator.compile(loss='binary_crossentropy', optimizer=optimizer)
  return generator

In [7]:
def get_discriminator(optimizer):
  discriminator=Sequential()
  discriminator.add(Dense(1024, input_dim=784, \
                      kernel_initializer=initializers.RandomNormal(stddev=0.02)))
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(Dropout(0.3))

  discriminator.add(Dense(512))
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(Dropout(0.3))

  discriminator.add(Dense(256))
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(Dropout(0.3))

  discriminator.add(Dense(1, activation='sigmoid'))
  discriminator.compile(loss='binary_crossentropy', optimizer=optimizer)
  return discriminator

In [8]:
def get_gan_network(discriminator, random_dim, generator, optimizer):
  discriminator.trainable=False
  gan_input=Input(shape=(random_dim,))
  x=generator(gan_input)
  gan_output=discriminator(x)
  gan=Model(inputs=gan_input, outputs=gan_output)
  gan.compile(loss='binary_crossentropy', optimizer=optimizer)
  return gan

In [9]:
def plot_generated_images(epoch, generator, examples=100, dim=(10,10), \
                          figsize=(10,10)):
  noise=np.random.normal(0,1,size=[examples, random_dim])
  generated_images=generator.predict(noise)
  generated_images=generated_images.reshape(examples, 28, 28)
  plt.figure(figsize=figsize)
  for i in range(generated_images.shape[0]):
    plt.subplot(dim[0], dim[1], i+1)
    plt.imshow(generated_images[i], interpolation='nearest', \
               cmap='gray_r')
    plt.axis('off')
  plt.tight_layout()
  plt.savefig('gan_generated_image_epoch_%d.png' %epoch)

In [11]:
def train(epochs=1, batch_size=128):
  x_train, y_train, x_test, y_test=load_mnist_data()
  batch_count = x_train.shape[0] / batch_size

  #2. Build our GAN netowrk
  adam = get_optimizer()
  generator = get_generator(adam)
  discriminator = get_discriminator(adam)
  gan = get_gan_network(discriminator, random_dim, generator, adam)
  # 3
  for e in range(1, epochs+1):
      print('-'*15, 'Epoch %d' % e, '-'*15)
      for _ in tqdm(range(int(batch_count))):

          # 4. Get a random set of input noise and images
          noise = np.random.normal(0, 1, size=[batch_size, random_dim])
          image_batch = x_train[np.random.randint(0, x_train.shape[0], \
                                                    size=batch_size)]

          # 5. Generate fake MNIST images
          generated_images = generator.predict(noise)
          X = np.concatenate([image_batch, generated_images])

          # 6. Labels for generated and real data
          y_dis = np.zeros(2*batch_size)
          # One-sided label smoothing
          y_dis[:batch_size] = 0.9

In [ ]:
train(20, 128)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


--------------- Epoch 1 ---------------


  0%|          | 0/468 [00:00<?, ?it/s]

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step 


...

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


 67%|██████▋   | 314/468 [01:12<00:31,  4.91it/s]

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


 67%|██████▋   | 315/468 [01:12<00:32,  4.68it/s]

1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step